<a href="https://colab.research.google.com/github/jford59/RT1/blob/main/HW2/RT2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Define modified AlexNet model
class ModifiedAlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedAlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 4 * 4, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

if __name__ == '__main__':
    # Load CIFAR-10 dataset and apply transformations with data augmentation
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),  # Random crop with padding
        transforms.RandomHorizontalFlip(),  # Random horizontal flip
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

    # Initialize the model, loss function, and optimizer
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    net = ModifiedAlexNet()
    net.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)  # Reduce LR by a factor of 0.1 every 30 epochs

    # Training the model
    for epoch in range(20):  # loop over the dataset multiple times
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        scheduler.step()  # Step the scheduler
        net.eval()
        correct = 0
        total = 0
        val_loss = 0.0
        with torch.no_grad():
            for data in testloader:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                val_loss += criterion(outputs, labels).item()
        print(f'Epoch [{epoch + 1}/20], '
              f'Training Loss: {running_loss / len(trainloader):.3f}, '
              f'Validation Loss: {val_loss / len(testloader):.3f}, '
              f'Validation Accuracy: {(100 * correct / total):.2f} %')

100%|██████████| 170498071/170498071 [00:03<00:00, 43151704.05it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1/20], Training Loss: 1.597, Validation Loss: 1.281, Validation Accuracy: 53.52 %
Epoch [2/20], Training Loss: 1.177, Validation Loss: 0.945, Validation Accuracy: 66.33 %
Epoch [3/20], Training Loss: 0.968, Validation Loss: 1.059, Validation Accuracy: 64.86 %
Epoch [4/20], Training Loss: 0.846, Validation Loss: 0.875, Validation Accuracy: 71.44 %
Epoch [5/20], Training Loss: 0.756, Validation Loss: 0.773, Validation Accuracy: 75.74 %
Epoch [6/20], Training Loss: 0.703, Validation Loss: 0.631, Validation Accuracy: 78.86 %
Epoch [7/20], Training Loss: 0.640, Validation Loss: 0.694, Validation Accuracy: 77.73 %
Epoch [8/20], Training Loss: 0.592, Validation Loss: 0.615, Validation Accuracy: 80.31 %
Epoch [9/20], Training Loss: 0.551, Validation Loss: 0.553, Validation Accuracy: 82.45 %
Epoch [10/20], Training Loss: 0.517, Validation Loss: 0.515, Validation Accuracy: 83.87 %
Epoch [11/20], Train